In [0]:
## Data Preparation
# List of functions for loading German Traffic Sign Data Set for (Training and Testing)
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)
    invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
    if invalid_path in sys.path:
        sys.path.remove(invalid_path)

In [0]:
## Import libraries need to be imported, Dont forget to update requirements.txt!
import os
import sys
IntialiseEnv()
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import random
import matplotlib.pyplot as plt
import import_ipynb
from Feature_Extraction.Segmentation import FeatureExtraction
from Feature_Extraction.Segmentation import FeatureExtractionWithoutGrayScaleConversion
from Feature_Extraction.Segmentation import augment_image
from Feature_Extraction.Segmentation import FeatureExtractionHSV

importing Jupyter notebook from D:\github\Traffic_Sign_Recognition_Detection\Source_Code\Feature_Extraction\Segmentation.ipynb


In [0]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(_dir):
    train_category_dir= os.path.join(training_dataset_dir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

In [0]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [0]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img= cv2.resize(img,(80,80))
    return img

In [0]:
def loadSampleImg(path):
    img=readImage(path)
    return img

In [0]:
def crop_image(img,x1,x2,y1,y2):
  img= img[x1:x2,y1:y2]
  img= cv2.resize(img,(80,80))
  # plt.imshow(img)
  return img

In [0]:
## Training DataSet Directory
training_dataset_dir = '../DataSet/Training_DataSet/Final_Training/Images'

def LoadTrainDataSet():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            ##### Crop image and resize
            
            list_=train_data_frame[train_data_frame['Filename'] == img_path[-15:]].index.tolist()
            x1=train_data_frame.iloc[list_[0]][3]
            x2=train_data_frame.iloc[list_[0]][5]
            y1=train_data_frame.iloc[list_[0]][4]
            y2=train_data_frame.iloc[list_[0]][6]
            img = crop_image(img,x1,x2,y1,y2)
            ####
            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array
        

In [0]:
def LoadTrainDataSetWithAugmentation():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            ##### Crop image and resize
            
            list_=train_data_frame[train_data_frame['Filename'] == img_path[-15:]].index.tolist()
            x1=train_data_frame.iloc[list_[0]][3]
            x2=train_data_frame.iloc[list_[0]][5]
            y1=train_data_frame.iloc[list_[0]][4]
            y2=train_data_frame.iloc[list_[0]][6]
            img = crop_image(img,x1,x2,y1,y2)
            ####
            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            #### for agumentation 
            random_number_probability=random.uniform(0, 1)
            if random_number_probability>=0.5:
                agumented_image=augment_image(img,0.7,11,5,2)
                img_features_= np.array(FeatureExtraction(agumented_image))[:,:,np.newaxis]
                train_labels_array.append(dir_img_label)
                train_image_array.append(img_features_)
            
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array

In [0]:
def LoadTrainDataSetWithAugmentationFeatureExtractionModified():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            ##### Crop image and resize
            
            list_=train_data_frame[train_data_frame['Filename'] == img_path[-15:]].index.tolist()
            x1=train_data_frame.iloc[list_[0]][3]
            x2=train_data_frame.iloc[list_[0]][5]
            y1=train_data_frame.iloc[list_[0]][4]
            y2=train_data_frame.iloc[list_[0]][6]
            img = crop_image(img,x1,x2,y1,y2)
            ####
            img_features = np.float32(FeatureExtractionWithoutGrayScaleConversion(img))
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            #### for agumentation 
            random_number_probability=random.uniform(0, 1)
            if random_number_probability>=0.5:
                agumented_image=augment_image(img,0.7,11,5,2)
                img_features_= np.float32(FeatureExtractionWithoutGrayScaleConversion(agumented_image))
                train_labels_array.append(dir_img_label)
                train_image_array.append(img_features_)
            
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array

In [0]:
def LoadTrainDataSetWithAugmentationHSV():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            ##### Crop image and resize
            
            list_=train_data_frame[train_data_frame['Filename'] == img_path[-15:]].index.tolist()
            x1=train_data_frame.iloc[list_[0]][3]
            x2=train_data_frame.iloc[list_[0]][5]
            y1=train_data_frame.iloc[list_[0]][4]
            y2=train_data_frame.iloc[list_[0]][6]
            img = crop_image(img,x1,x2,y1,y2)
            ####
            img_features = np.float32(FeatureExtractionHSV(img))
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            #### for agumentation 
            random_number_probability=random.uniform(0, 1)
            if random_number_probability>=0.5:
                agumented_image=augment_image(img,0.7,11,5,2)
                img_features_= np.float32(FeatureExtractionHSV(agumented_image))
                train_labels_array.append(dir_img_label)
                train_image_array.append(img_features_)
            
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array

In [0]:
test_dir='../DataSet/Testing_DataSet/Final_Test/Images/'
labels_dir='../DataSet/Testing_DataSet/GT-final_test.csv'

In [0]:
def LoadTestDataSet():
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels


In [0]:
def LoadTestDataSetForMain():
    test_dir='DataSet/Testing_DataSet/Final_Test/Images/'
    labels_dir='DataSet/Testing_DataSet/GT-final_test.csv'
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        ##### Crop image and resize
            
        # list_=test_Data_frame[test_Data_frame['Filename'] == img_path[-15:]].index.tolist()
        x1=test_Data_frame.iloc[list_[0]][3]
        x2=test_Data_frame.iloc[list_[0]][5]
        y1=test_Data_frame.iloc[list_[0]][4]
        y2=test_Data_frame.iloc[list_[0]][6]
        img = crop_image(img,x1,x2,y1,y2)
        ######
        img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels

In [0]:
def LoadTestDataSetFeatureExtractionModified():
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        ##### Crop image and resize
            
        # list_=test_Data_frame[test_Data_frame['Filename'] == img_path[-15:]].index.tolist()
        x1=test_Data_frame.iloc[list_[0]][3]
        x2=test_Data_frame.iloc[list_[0]][5]
        y1=test_Data_frame.iloc[list_[0]][4]
        y2=test_Data_frame.iloc[list_[0]][6]
        img = crop_image(img,x1,x2,y1,y2)
        ######
        img_features = np.array(FeatureExtractionWithoutGrayScaleConversion(img))
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels


In [0]:
def LoadTestDataSetFeatureExtractionModifiedForMain():
    test_dir='DataSet/Testing_DataSet/Final_Test/Images/'
    labels_dir='DataSet/Testing_DataSet/GT-final_test.csv'
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        ##### Crop image and resize
            
        # list_=test_Data_frame[test_Data_frame['Filename'] == img_path[-15:]].index.tolist()
        x1=test_Data_frame.iloc[list_[0]][3]
        x2=test_Data_frame.iloc[list_[0]][5]
        y1=test_Data_frame.iloc[list_[0]][4]
        y2=test_Data_frame.iloc[list_[0]][6]
        img = crop_image(img,x1,x2,y1,y2)
        ######
        img_features = np.array(FeatureExtractionWithoutGrayScaleConversion(img))
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels

In [0]:
def LoadTestDataSetHSV():
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        ##### Crop image and resize
            
        # list_=test_Data_frame[test_Data_frame['Filename'] == img_path[-15:]].index.tolist()
        x1=test_Data_frame.iloc[list_[0]][3]
        x2=test_Data_frame.iloc[list_[0]][5]
        y1=test_Data_frame.iloc[list_[0]][4]
        y2=test_Data_frame.iloc[list_[0]][6]
        img = crop_image(img,x1,x2,y1,y2)
        ######
        img_features = np.float32(FeatureExtractionHSV(img))
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels

In [0]:
def LoadTestDataSetHSVForMain():
    test_dir='DataSet/Testing_DataSet/Final_Test/Images/'
    labels_dir='DataSet/Testing_DataSet/GT-final_test.csv'
    test_image_array=[]
    test_image_labels=[]
    test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        print("Directory: ", _dir)
        list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
        dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
        
        img = readImage(test_dir+_dir)
        ##### Crop image and resize
            
        # list_=test_Data_frame[test_Data_frame['Filename'] == img_path[-15:]].index.tolist()
        x1=test_Data_frame.iloc[list_[0]][3]
        x2=test_Data_frame.iloc[list_[0]][5]
        y1=test_Data_frame.iloc[list_[0]][4]
        y2=test_Data_frame.iloc[list_[0]][6]
        img = crop_image(img,x1,x2,y1,y2)
        ######
        img_features = np.float32(FeatureExtractionHSV(img))
        test_image_array.append(img_features)
        test_image_labels.append(dir_img_label)
        
    test_image_array=np.stack(test_image_array, axis=0)
    return   test_image_array, test_image_labels